# AutoGluon Object Detection Tutorial

### Prerequisites

SageMaker Ground Truth로 레이블링 수행 완료 후 output.manifest를 로컬로 복사

![sagemaker-gt](./script/sagemaker-gt.png)

### References
- AutoGluon Multimodal - Quick Start: https://auto.gluon.ai/stable/tutorials/multimodal/multimodal_prediction/multimodal-quick-start.html

### Download dataset

You need to prepare your onw dataset

In [ ]:
task = 'objdetection-pikachu'
!rm *.json
!rm -rf AutogluonModels tmp
!rm -rf {task} && mkdir -p {task}
!curl -L "https://universe.roboflow.com/ds/xZJGldQLjS?key=B56tyPNKRM" > roboflow.zip; unzip -q roboflow.zip -d {task}/images; rm roboflow.zip

### Create Directory
AutoGluon MultiModalPredictor의 Object Detection은 아래와 같은 디렉토리로 설정하는 것을 추천
```
├── [YOUR-TASK-DIR]
│   ├── annotations: coco 포맷의 json annotation 파일이 위치한 경로
│   └── images: raw 이미지 경로
```

본 샘플 코드의 예시는 아래와 같다.
```
├── objdetection-pikachu
│   ├── annotations
│   └── images
│       ├── train
│       ├── valid
│       └── test
```


In [ ]:
dataset_path = f"{task}/images"
annotations_path = f"{task}/annotations"
!mkdir -p {annotations_path}

### Convert SageMaker Ground Truth manifest file to COCO-formatted json file
- Step 1: SageMaker groundth truth의 manifest output 파일을 COCO 포맷으로 변환 (필요 시 utils.py 참조하여 수정할 것)
- Step 2: COCO 포맷 파일을 데이터셋 폴더로 복사

In [ ]:
import shutil
from script.utils import convert_bbox_manifest, show_detection_result
dataset_type = ['train', 'valid', 'test']
json_path_dict = {}

for d in dataset_type:
    manifest_path = f"manifest/pikachu-{d}-output.manifest"
    job_name = f"objdetect-pikachu-gt-{d}-job"
    output_coco_json_path = f"pikachu-{d}-coco.json"
    convert_bbox_manifest(manifest_path, job_name, output_coco_json_path, d)
    json_path = f"{annotations_path}/{output_coco_json_path}"
    shutil.copyfile(output_coco_json_path, json_path)
    json_path_dict[d] = json_path

train_path = json_path_dict['train']
valid_path = json_path_dict['valid']
test_path = json_path_dict['test']

print(train_path, valid_path, test_path)

https://github.com/open-mmlab/mmdetection/tree/master/configs/yolox

### Create MultiModalPredictor
- `problem_type="object_detection"` 으로 지정
- `sample_data_path`는 객체(object)의 범주를 파악하기 위해 필요하며 COCO 포맷 json에 명시되어 있음
- `model.mmdet_image.checkpoint_name`에서 객체 검출에 쓰일 모델 이름을 지정. 직접 모델을 지정하기 어렵다면 preset으로도 지정 가능 (예: `medium_quality, high_quality, best_quality`)

In [ ]:
from autogluon.multimodal import MultiModalPredictor
# Init predictor
import uuid
#presets = "medium_quality"
checkpoint_name = "yolox_l_8x8_300e_coco"

model_path = f"./tmp/{uuid.uuid4().hex}-{task}-save"
predictor = MultiModalPredictor(
    hyperparameters={
        "model.mmdet_image.checkpoint_name": checkpoint_name,
        "optimization.val_metric": "map",
    },
    problem_type="object_detection",
    sample_data_path=train_path,
    #presets=presets,
    path=model_path
)

### Training
- More parameters: https://auto.gluon.ai/dev/tutorials/multimodal/advanced_topics/customization.html

In [ ]:
import time
start = time.time()
predictor.fit(
    train_data=train_path,
    tuning_data=valid_path,
    hyperparameters={
        "optimization.learning_rate": 3e-5,
        "env.per_gpu_batch_size": 8,  # decrease it when model is large
        "optimization.max_epochs": 10,  # max number of training epochs, note that we may early stop before this based on validation setting
        "optimization.check_val_every_n_epoch": 2,  # Do k validation each 2 epochs
        "optimization.patience": 3,  # Early stop after k consective validations are not the best
    },
)  # Fit
end = time.time()
print("This finetuning takes %.2f seconds." % (end - start))

### Evaluation

In [ ]:
predictor.evaluate(valid_path)

### Load model

In [ ]:
import glob

In [ ]:
from glob import glob
test_path =f"{task}/images/test" 

test_files = glob(f'{test_path}/*.jpg') + glob(f'{test_path}/*.jpeg') + glob(f'{test_path}/*.png')

### Prediction

In [ ]:
from glob import glob
test_path = f"{task}/images/test" 
test_files = glob(f'{test_path}/*.jpg') + glob(f'{test_path}/*.jpeg') + glob(f'{test_path}/*.png')

In [ ]:
test_idx = 0
img_path = test_files[test_idx]
img_result = predictor.predict(img_path).iloc[0]
print(img_result)
show_detection_result(img_path, img_result, 0.5)